In [2]:
## This will serve as a consolidated notebook between the two sets of GI experiments done for Herken2023 to call 
## genetic interactions 

In [1]:
## Import modules

import numpy as np
import scipy as sp
import pandas as pd
from scipy import optimize
from scipy import stats

In [2]:
## Call functions

####Analysis pipeline####

## ALL THAT FOLLOWS IN THIS CELL WAS CREATED BY MAX HORLBECK

#version that only filters based on cycledCol as many sgRNAs have median rep of 0 at the end
def calcLog2e_cycledonly(t0Col, cycledCol, doublesTable, filterThreshold = 1.0, pseudocount = 1.0, doublingDifferences = 1.0):
    meanCounts = pd.concat((cycledCol.groupby(doublesTable['name_a']).agg(np.median),cycledCol.groupby(doublesTable['name_b']).agg(np.median)),axis=1, keys=['a','b'])
    
    sgsToFilter = set(meanCounts.loc[meanCounts.loc[:,'b'] < filterThreshold].index).union(set(meanCounts.loc[meanCounts.loc[:,'a'] < filterThreshold].index))
    doublesTable_filt = doublesTable.loc[doublesTable.apply(lambda row: row['name_a'] not in sgsToFilter and row['name_b'] not in sgsToFilter, axis=1)]
    print(str(len(doublesTable_filt)) + ' pairs of ' + str(len(doublesTable)) + ' passing filter')
    
    countsRatio = (t0Col.loc[doublesTable_filt.index] + pseudocount).sum()*1.0/(cycledCol.loc[doublesTable_filt.index] + pseudocount).sum()
    log2es = np.log2((cycledCol.loc[doublesTable_filt.index] + pseudocount)/(t0Col.loc[doublesTable_filt.index] + pseudocount)/countsRatio)

    doubleNegatives = doublesTable.apply(lambda row: row['gene_a'] == 'negative' and row['gene_b'] == 'negative', axis=1)

    log2es -= log2es.loc[doubleNegatives].median()

    log2es /= doublingDifferences
    
    return log2es

#for a specified variable position and sgRNA, get single phenotypes, double phenotypes, and optionally single phenotype std dev.
def getXYB(sgRNA, singlePhenotypes, phenotypeMatrix, variablePosition, fixedPosition, returnXerr=False):
    if not returnXerr:
        return singlePhenotypes[variablePosition+'.mean'], \
            phenotypeMatrix.loc[sgRNA,:] if fixedPosition == 'a' else phenotypeMatrix.loc[:,sgRNA], \
            singlePhenotypes.loc[sgRNA, fixedPosition +'.mean']
    else:
        return singlePhenotypes[variablePosition+'.mean'], \
            phenotypeMatrix.loc[sgRNA,:] if fixedPosition == 'a' else phenotypeMatrix.loc[:,sgRNA], \
            singlePhenotypes.loc[sgRNA, fixedPosition +'.mean'], singlePhenotypes[variablePosition+'.std']
        
#convert phenotypes into square matrix
def generatePhenotypeMatrix(phenotypes):
    numSingles = int(np.sqrt(len(phenotypes)))
    phenotypeMatrix = np.zeros((numSingles,numSingles))
    singlesTable = []
    for i, (sgPair, counts) in enumerate(phenotypes.sort_index().iteritems()):
        phenotypeMatrix[int(i/numSingles), i%numSingles] = counts
        if i%numSingles == 0:
            singlesTable.append(sgPair.split(':')[0])

    phenotypeMatrix = pd.DataFrame(phenotypeMatrix, index=singlesTable, columns=singlesTable)
    singlesTable = pd.DataFrame([s.split('_')[0] for s in singlesTable], index=singlesTable, columns=['gene'])
    
    singlePhenotypes = pd.concat((phenotypeMatrix.loc[singlesTable['gene'] == 'non-targeting',:].apply(np.nanmean, axis=0), 
                                  phenotypeMatrix.loc[singlesTable['gene'] == 'non-targeting',:].apply(np.nanstd, axis=0), 
                                  phenotypeMatrix.loc[:, singlesTable['gene'] == 'non-targeting'].apply(np.nanmean, axis=1),
                                 phenotypeMatrix.loc[:, singlesTable['gene'] == 'non-targeting'].apply(np.nanstd, axis=1)), 
                                 axis=1, keys=['b.mean','b.std','a.mean','a.std'])
    
    return phenotypeMatrix, singlesTable, singlePhenotypes

def abbaAveragePhenotypes(phenotypeMatrix, singlesTable):
	phenotypeMatrix_abba = (phenotypeMatrix + phenotypeMatrix.T) / 2

	singlePhenotypes_abba = pd.concat((phenotypeMatrix_abba.loc[singlesTable['gene'] == 'non-targeting',:].apply(np.nanmean, axis=0), 
                                  phenotypeMatrix_abba.loc[singlesTable['gene'] == 'non-targeting',:].apply(np.nanstd, axis=0), 
                                  phenotypeMatrix_abba.loc[:, singlesTable['gene'] == 'non-targeting'].apply(np.nanmean, axis=1),
                                  phenotypeMatrix_abba.loc[:, singlesTable['gene'] == 'non-targeting'].apply(np.nanstd, axis=1)), 
                                 axis=1, keys=['b.mean','b.std','a.mean','a.std'])

	return phenotypeMatrix_abba, singlePhenotypes_abba


#calculate epistasis interactions, optionally z-standardizing based on negative controls
def calculateInteractions(phenotypeMatrix, singlePhenotypes, singlesTable, fitFunction, zstandardize=True):
    emap1 = pd.DataFrame(np.zeros(phenotypeMatrix.shape), index=phenotypeMatrix.index, columns=phenotypeMatrix.columns)
    variablePosition, fixedPosition = 'a','b'
    for i, sgRNA in enumerate(phenotypeMatrix.index):
        xdata, ydata, bdata = getXYB(sgRNA, singlePhenotypes, phenotypeMatrix, variablePosition, fixedPosition)
        
        fit = fitFunction(xdata, ydata, bdata)
        epistasis = ydata - fit(xdata)

        if zstandardize:
	        emap1.loc[sgRNA,:] = epistasis / epistasis.loc[singlesTable['gene'] == 'non-targeting'].std()
       	else:
	        emap1.loc[sgRNA,:] = epistasis 

    emap2 = pd.DataFrame(np.zeros(phenotypeMatrix.shape), index=phenotypeMatrix.index, columns=phenotypeMatrix.columns)
    variablePosition, fixedPosition = 'b','a'
    for i, sgRNA in enumerate(phenotypeMatrix.index):
        xdata, ydata, bdata = getXYB(sgRNA, singlePhenotypes, phenotypeMatrix, variablePosition, fixedPosition)
        
        fit = fitFunction(xdata, ydata, bdata)
        epistasis = ydata - fit(xdata)

        if zstandardize:
	        emap2.loc[sgRNA,:] = epistasis / epistasis.loc[singlesTable['gene'] == 'non-targeting'].std()
       	else:
	        emap2.loc[sgRNA,:] = epistasis 

    emap12 = (emap1+emap2)/2
    
    emap_ave = (emap12 + emap12.T) / 2
    
    return emap1, emap2, emap_ave

#calculate all pairwise intra-sgRNA or intra-gene correlations
def calculateCorrelationMatrix(matrix, diagNull=True):
    corrMatrix = pd.DataFrame(np.corrcoef(matrix), index=matrix.index, columns=matrix.columns)
    
    if diagNull:
        for i in range(len(corrMatrix)):
            corrMatrix.iloc[i,i] = np.nan
            
    return corrMatrix

#find correlations between sgRNAs targeting the same gene and negative controls
def calculateIntrageneCorrelation(sgCorrMatrix, singlePhenotypes, singlesTable):
    sameGeneCorrTups = []
    negCorrTups = []
    for gene, sgs in singlesTable.groupby('gene'):
        for i, (sg1, row) in enumerate(sgCorrMatrix.loc[sgs.index, sgs.index].iterrows()):
            for j, (sg2, val) in enumerate(row.iteritems()):
                if i>j:
                    if gene != 'negative':
                        sameGeneCorrTups.append((sg1, sg2, 
                                                 singlePhenotypes.loc[sg1,['a.mean','b.mean']].mean(), 
                                                 singlePhenotypes.loc[sg2,['a.mean','b.mean']].mean(),
                                                val))
                    else:
                        negCorrTups.append((sg1, sg2, 
                                                 singlePhenotypes.loc[sg1,['a.mean','b.mean']].mean(), 
                                                 singlePhenotypes.loc[sg2,['a.mean','b.mean']].mean(),
                                                val))
                        
    return sameGeneCorrTups, negCorrTups


#generate a gene map by averaging sgRNA epistasis
def generateGeneMap(emap_sgRNA, singlesTable):
    emap_gene = pd.DataFrame(np.zeros((len(set(singlesTable['gene'])),len(set(singlesTable['gene'])))), index = sorted(set(singlesTable['gene'])), columns = sorted(set(singlesTable['gene'])))
    for gene_a, rowgroup in emap_sgRNA.groupby(singlesTable['gene']):
        for gene_b, colgroup in rowgroup.groupby(singlesTable['gene'], axis=1):
            emap_gene.loc[gene_a, gene_b] = colgroup.sum().sum() / (colgroup.shape[0] * colgroup.shape[1])
            
    return emap_gene

### fit functions for calculating interactions and plotting
def linearFitForceIntercept(xdata, ydata, bdata):
    m1 = optimize.fmin(lambda m, x, y: ((m*x + bdata - y)**2).sum(), x0=0.1, args=(xdata, ydata), disp=0)[0]
    
    return lambda x1: m1*np.array(x1) + bdata

def quadFitForceIntercept(xdata, ydata, bdata):
    m1 = optimize.fmin(lambda m, x, y: ((m[0]*(x**2) + m[1]*x + bdata - y)**2).sum(), x0=[0.1,0.1], args=(xdata, ydata), disp=0)
    
    return lambda x1: m1[0]*(np.array(x1)**2) + m1[1]*np.array(x1) + bdata


In [3]:
## This updated function corrects a mistake in Max's "calcLog2e_cycledonly" function, which while ultimately not 
## affecting the data is important not to continue to propigate

def calcLog2e_cycledonly_corrected(t0Col, cycledCol, doublesTable, filterThreshold = 1.0, pseudocount = 1.0, doublingDifferences = 1.0):
    meanCounts = pd.concat((cycledCol.groupby(doublesTable['name_a']).agg(np.median),cycledCol.groupby(doublesTable['name_b']).agg(np.median)),axis=1, keys=['a','b'])
    
    sgsToFilter = set(meanCounts.loc[meanCounts.loc[:,'b'] < filterThreshold].index).union(set(meanCounts.loc[meanCounts.loc[:,'a'] < filterThreshold].index))
    doublesTable_filt = doublesTable.loc[doublesTable.apply(lambda row: row['name_a'] not in sgsToFilter and row['name_b'] not in sgsToFilter, axis=1)]
    print(str(len(doublesTable_filt)) + ' pairs of ' + str(len(doublesTable)) + ' passing filter')
    
    countsRatio = (t0Col.loc[doublesTable_filt.index] + pseudocount).sum()*1.0/(cycledCol.loc[doublesTable_filt.index] + pseudocount).sum()
    log2es = np.log2(((cycledCol.loc[doublesTable_filt.index] + pseudocount)/(t0Col.loc[doublesTable_filt.index] + pseudocount))*countsRatio)

    doubleNegatives = doublesTable.apply(lambda row: row['gene_a'] == 'negative' and row['gene_b'] == 'negative', axis=1)

    log2es -= log2es.loc[doubleNegatives].median()

    log2es /= doublingDifferences
    
    return log2es


In [4]:
## Import Data

## The three files referenced in defining variables below can be found at: 
## https://ucsf.box.com/s/nuwov4kgb55mqfrr7j5a216ot78f4uxn
## and should be used to define these variables accordingly:

## expt1_countsFile = pd.read_csv('../GI_expt1_counts.txt',sep='\t').set_index('double')
## expt2_countsFile = pd.read_csv('../GI_expt2_counts.txt',sep='\t').set_index('double')
## doublesLibrary = pd.read_csv('../dualsgrna_alignedlibrary.txt',sep='\t').set_index('double')

expt1_countsFile = pd.read_csv('/Users/benh/Desktop/UCSF_Tetrad_Program/Gilbert_Lab/DDRmap/ddrGI_K562_results/Counts_Files_Processed/ddrGI_K562_CountsMaster.txt',sep='\t').set_index('double')
expt2_countsFile = pd.read_csv('/Users/benh/Desktop/UCSF_Tetrad_Program/Gilbert_Lab/gi2/counts/counts_master.txt').set_index('double')

doublesLibrary = pd.read_csv('/Users/benh/Desktop/UCSF_Tetrad_Program/Gilbert_Lab/DDRmap/ddrGI_Libraries/ddrGI_doubles.txt',sep='\t').set_index('double')


In [5]:
## define growthScores for normalizing data by each conditions number of cell population doublings over the course 
## of the experiment
growthScores = {
    'expt1_rep1_dmso' : 9.632926829,
    'expt1_rep1_atr' : 4.76097561,
    'expt1_rep2_dmso' : 9.559756098,
    'expt1_rep2_atr' : 4.807317073,
    'expt2_rep1_dmso' : 5.761137265,
    'expt2_rep1_etop' : 2.251010599,
    'expt2_rep1_keto' : 4.173335676,
    'expt2_rep2_dmso' : 6.324829553,
    'expt2_rep2_etop' : 2.211941293,
    'expt2_rep2_keto' : 4.807317073}

In [6]:
## Calculate expt1_dmso replicate phenotypes

expt1_dmso_r1_phenotypes = calcLog2e_cycledonly(expt1_countsFile['Rep1_T0'],
                                   expt1_countsFile['Rep1_UT'],
                                   doublesLibrary,
                                   filterThreshold=35,
                                   pseudocount=10,
                                   doublingDifferences=growthScores['expt1_rep1_dmso'])

expt1_dmso_r2_phenotypes = calcLog2e_cycledonly(expt1_countsFile['Rep2_T0'],
                                   expt1_countsFile['Rep2_UT'],
                                   doublesLibrary,
                                   filterThreshold=35,
                                   pseudocount=10,
                                   doublingDifferences=growthScores['expt1_rep2_dmso'])

400689 pairs of 407044 passing filter
398161 pairs of 407044 passing filter


In [7]:
## Calculate expt1_atr replicate phenotypes

expt1_atr_r1_phenotypes = calcLog2e_cycledonly(expt1_countsFile['Rep1_T0'],
                                   expt1_countsFile['Rep1_DRUG'],
                                   doublesLibrary,
                                   filterThreshold=35,
                                   pseudocount=10,
                                   doublingDifferences=growthScores['expt1_rep1_atr'])

expt1_atr_r2_phenotypes = calcLog2e_cycledonly(expt1_countsFile['Rep2_T0'],
                                   expt1_countsFile['Rep2_DRUG'],
                                   doublesLibrary,
                                   filterThreshold=35,
                                   pseudocount=10,
                                   doublingDifferences=growthScores['expt1_rep2_atr'])

370881 pairs of 407044 passing filter
357604 pairs of 407044 passing filter


In [8]:
## Calculate expt2_dmso replicate phenotypes

expt2_dmso_r1_phenotypes = calcLog2e_cycledonly(expt2_countsFile['t0_r1'],
                                   expt2_countsFile['dmso_r1'],
                                   doublesLibrary,
                                   filterThreshold=35,
                                   pseudocount=10,
                                   doublingDifferences=growthScores['expt2_rep1_dmso'])

expt2_dmso_r2_phenotypes = calcLog2e_cycledonly(expt2_countsFile['t0_r2'],
                                   expt2_countsFile['dmso_r2'],
                                   doublesLibrary,
                                   filterThreshold=35,
                                   pseudocount=10,
                                   doublingDifferences=growthScores['expt2_rep2_dmso'])

403225 pairs of 407044 passing filter
401956 pairs of 407044 passing filter


In [10]:
## Calculate expt2_etop replicate phenotypes

expt2_etop_r1_phenotypes = calcLog2e_cycledonly(expt2_countsFile['t0_r1'],
                                   expt2_countsFile['etop_r1'],
                                   doublesLibrary,
                                   filterThreshold=35,
                                   pseudocount=10,
                                   doublingDifferences=growthScores['expt2_rep1_etop'])

expt2_etop_r2_phenotypes = calcLog2e_cycledonly(expt2_countsFile['t0_r2'],
                                   expt2_countsFile['etop_r2'],
                                   doublesLibrary,
                                   filterThreshold=35,
                                   pseudocount=10,
                                   doublingDifferences=growthScores['expt2_rep2_etop'])

404496 pairs of 407044 passing filter
404496 pairs of 407044 passing filter


In [9]:
## Calculate expt2_keto replicate phenotypes

expt2_keto_r1_phenotypes = calcLog2e_cycledonly(expt2_countsFile['t0_r1'],
                                   expt2_countsFile['keto_r1'],
                                   doublesLibrary,
                                   filterThreshold=35,
                                   pseudocount=10,
                                   doublingDifferences=growthScores['expt2_rep1_keto'])

expt2_keto_r2_phenotypes = calcLog2e_cycledonly(expt2_countsFile['t0_r2'],
                                   expt2_countsFile['keto_r2'],
                                   doublesLibrary,
                                   filterThreshold=35,
                                   pseudocount=10,
                                   doublingDifferences=growthScores['expt2_rep2_keto'])

399424 pairs of 407044 passing filter
396900 pairs of 407044 passing filter


In [11]:
## Repave expt1_dmso replicates to have the same list of threshold passed sgRNAs

sgIntersect =set(expt1_dmso_r1_phenotypes.index).intersection(expt1_dmso_r2_phenotypes.index)
expt1_dmso_repave = ((expt1_dmso_r1_phenotypes + expt1_dmso_r2_phenotypes) / 2).loc[sgIntersect]
print(len(expt1_dmso_repave))

/var/folders/wk/hds_px4x4qg3m4rvxqhrb0wm0000gp/T/ipykernel_89168/3494971632.py:4: FutureWarning: Passing a set as an indexer is deprecated and will raise in a future version. Use a list instead.
  expt1_dmso_repave = ((expt1_dmso_r1_phenotypes + expt1_dmso_r2_phenotypes) / 2).loc[sgIntersect]


398161


In [12]:
## Repave expt1_atr replicates to have the same list of threshold passed sgRNAs

sgIntersect =set(expt1_atr_r1_phenotypes.index).intersection(expt1_atr_r2_phenotypes.index)
expt1_atr_repave = ((expt1_atr_r1_phenotypes + expt1_atr_r2_phenotypes) / 2).loc[sgIntersect]
print(len(expt1_atr_repave))

/var/folders/wk/hds_px4x4qg3m4rvxqhrb0wm0000gp/T/ipykernel_89168/3623308769.py:4: FutureWarning: Passing a set as an indexer is deprecated and will raise in a future version. Use a list instead.
  expt1_atr_repave = ((expt1_atr_r1_phenotypes + expt1_atr_r2_phenotypes) / 2).loc[sgIntersect]


357604


In [13]:
## Repave expt2_dmso replicates to have the same list of threshold passed sgRNAs

sgIntersect =set(expt2_dmso_r1_phenotypes.index).intersection(expt2_dmso_r2_phenotypes.index)
expt2_dmso_repave = ((expt2_dmso_r1_phenotypes + expt2_dmso_r2_phenotypes) / 2).loc[sgIntersect]
print(len(expt2_dmso_repave))

/var/folders/wk/hds_px4x4qg3m4rvxqhrb0wm0000gp/T/ipykernel_89168/4191647971.py:4: FutureWarning: Passing a set as an indexer is deprecated and will raise in a future version. Use a list instead.
  expt2_dmso_repave = ((expt2_dmso_r1_phenotypes + expt2_dmso_r2_phenotypes) / 2).loc[sgIntersect]


401956


In [14]:
## Repave expt2_etop replicates to have the same list of threshold passed sgRNAs

sgIntersect =set(expt2_etop_r1_phenotypes.index).intersection(expt2_etop_r2_phenotypes.index)
expt2_etop_repave = ((expt2_etop_r1_phenotypes + expt2_etop_r2_phenotypes) / 2).loc[sgIntersect]
print(len(expt2_etop_repave))

/var/folders/wk/hds_px4x4qg3m4rvxqhrb0wm0000gp/T/ipykernel_89168/1857160102.py:4: FutureWarning: Passing a set as an indexer is deprecated and will raise in a future version. Use a list instead.
  expt2_etop_repave = ((expt2_etop_r1_phenotypes + expt2_etop_r2_phenotypes) / 2).loc[sgIntersect]


404496


In [15]:
## Repave expt2_keto replicates to have the same list of threshold passed sgRNAs

sgIntersect =set(expt2_keto_r1_phenotypes.index).intersection(expt2_keto_r2_phenotypes.index)
expt2_keto_repave = ((expt2_keto_r1_phenotypes + expt2_keto_r2_phenotypes) / 2).loc[sgIntersect]
print(len(expt2_keto_repave))

/var/folders/wk/hds_px4x4qg3m4rvxqhrb0wm0000gp/T/ipykernel_89168/4058603832.py:4: FutureWarning: Passing a set as an indexer is deprecated and will raise in a future version. Use a list instead.
  expt2_keto_repave = ((expt2_keto_r1_phenotypes + expt2_keto_r2_phenotypes) / 2).loc[sgIntersect]


396900


In [16]:
## Make phenotype matrices for all replicate averaged maps

dmso1_phenotypes, dmso1_singles_table, dmso1_singles_phenotypes = generatePhenotypeMatrix(expt1_dmso_repave)
atr_phenotypes, atr_singles_table, atr_singles_phenotypes = generatePhenotypeMatrix(expt1_atr_repave)
dmso2_phenotypes, dmso2_singles_table, dmso2_singles_phenotypes = generatePhenotypeMatrix(expt2_dmso_repave)
etop_phenotypes, etop_singles_table, etop_singles_phenotypes = generatePhenotypeMatrix(expt2_etop_repave)
keto_phenotypes, keto_singles_table, keto_singles_phenotypes = generatePhenotypeMatrix(expt2_keto_repave)

/var/folders/wk/hds_px4x4qg3m4rvxqhrb0wm0000gp/T/ipykernel_89168/803172525.py:42: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for i, (sgPair, counts) in enumerate(phenotypes.sort_index().iteritems()):
/var/folders/wk/hds_px4x4qg3m4rvxqhrb0wm0000gp/T/ipykernel_89168/803172525.py:42: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for i, (sgPair, counts) in enumerate(phenotypes.sort_index().iteritems()):
/var/folders/wk/hds_px4x4qg3m4rvxqhrb0wm0000gp/T/ipykernel_89168/803172525.py:42: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for i, (sgPair, counts) in enumerate(phenotypes.sort_index().iteritems()):
/var/folders/wk/hds_px4x4qg3m4rvxqhrb0wm0000gp/T/ipykernel_89168/803172525.py:42: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for i, (sgPair, counts) in enumerate(ph

In [17]:
## ABBA normalize data

dmso1_phenotypes_abba, dmso1_singles_phenotypes_abba = abbaAveragePhenotypes(dmso1_phenotypes, dmso1_singles_table)
atr_phenotypes_abba, atr_singles_phenotypes_abba = abbaAveragePhenotypes(atr_phenotypes, atr_singles_table)
dmso2_phenotypes_abba, dmso2_singles_phenotypes_abba = abbaAveragePhenotypes(dmso2_phenotypes, dmso2_singles_table)
etop_phenotypes_abba, etop_singles_phenotypes_abba = abbaAveragePhenotypes(etop_phenotypes, etop_singles_table)
keto_phenotypes_abba, keto_singles_phenotypes_abba = abbaAveragePhenotypes(keto_phenotypes, keto_singles_table)

In [38]:
## Calculate GIs

dmso1_map_1, dmso1_map_2, dmso1_map_guide = calculateInteractions(dmso1_phenotypes_abba, dmso1_singles_phenotypes_abba, dmso1_singles_table, quadFitForceIntercept, zstandardize=True)
atr_map_1, atr_map_2, atr_map_guide = calculateInteractions(atr_phenotypes_abba, atr_singles_phenotypes_abba, atr_singles_table, quadFitForceIntercept, zstandardize=True)
dmso2_map_1, dmso2_map_2, dmso2_map_guide = calculateInteractions(dmso2_phenotypes_abba, dmso2_singles_phenotypes_abba, dmso2_singles_table, quadFitForceIntercept, zstandardize=True)
etop_map_1, etop_map_2, etop_map_guide = calculateInteractions(etop_phenotypes_abba, etop_singles_phenotypes_abba, etop_singles_table, quadFitForceIntercept, zstandardize=True)
keto_map_1, keto_map_2, keto_map_guide = calculateInteractions(keto_phenotypes_abba, keto_singles_phenotypes_abba, keto_singles_table, quadFitForceIntercept, zstandardize=True)



In [44]:
## Calculate gene level maps

dmso1_map = generateGeneMap(dmso1_map_guide, dmso1_singles_table)
atr_map = generateGeneMap(atr_map_guide, atr_singles_table)
dmso2_map = generateGeneMap(dmso2_map_guide, dmso2_singles_table)
etop_map = generateGeneMap(etop_map_guide, etop_singles_table)
keto_map = generateGeneMap(keto_map_guide, keto_singles_table)

In [51]:
## Repave all maps to have the same index

intersect = set(dmso1_map.index).intersection(atr_map.index).intersection(dmso2_map.index).intersection(etop_map.index).intersection(keto_map.index)
dmso1_map = dmso1_map.loc[intersect,intersect]
atr_map = atr_map.loc[intersect,intersect]
dmso2_map = dmso2_map.loc[intersect,intersect]
etop_map = etop_map.loc[intersect,intersect]
keto_map = keto_map.loc[intersect,intersect]


/var/folders/wk/hds_px4x4qg3m4rvxqhrb0wm0000gp/T/ipykernel_60078/3407015454.py:4: FutureWarning: Passing a set as an indexer is deprecated and will raise in a future version. Use a list instead.
  dmso1_map = dmso1_map.loc[intersect,intersect]
/var/folders/wk/hds_px4x4qg3m4rvxqhrb0wm0000gp/T/ipykernel_60078/3407015454.py:5: FutureWarning: Passing a set as an indexer is deprecated and will raise in a future version. Use a list instead.
  atr_map = atr_map.loc[intersect,intersect]
/var/folders/wk/hds_px4x4qg3m4rvxqhrb0wm0000gp/T/ipykernel_60078/3407015454.py:6: FutureWarning: Passing a set as an indexer is deprecated and will raise in a future version. Use a list instead.
  dmso2_map = dmso2_map.loc[intersect,intersect]
/var/folders/wk/hds_px4x4qg3m4rvxqhrb0wm0000gp/T/ipykernel_60078/3407015454.py:7: FutureWarning: Passing a set as an indexer is deprecated and will raise in a future version. Use a list instead.
  etop_map = etop_map.loc[intersect,intersect]
/var/folders/wk/hds_px4x4qg3m4

In [55]:
## Write gene-level data

dmso1_map.to_excel('/Users/benh/Desktop/GI_data/gene_maps/dmso1_map.xlsx')
atr_map.to_excel('/Users/benh/Desktop/GI_data/gene_maps/atr_map.xlsx')
dmso2_map.to_excel('/Users/benh/Desktop/GI_data/gene_maps/dmso2_map.xlsx')
etop_map.to_excel('/Users/benh/Desktop/GI_data/gene_maps/etop_map.xlsx')
keto_map.to_excel('/Users/benh/Desktop/GI_data/gene_maps/keto_map.xlsx')

In [18]:
## Write phenotype matrixes

dmso1_phenotypes_abba.to_excel('/Users/benh/Desktop/GI_data/phenotype_matrices/dmso1_phenotypes.xlsx')
atr_phenotypes_abba.to_excel('/Users/benh/Desktop/GI_data/phenotype_matrices/atr_phenotypes.xlsx')
dmso2_phenotypes_abba.to_excel('/Users/benh/Desktop/GI_data/phenotype_matrices/dmso2_phenotypes.xlsx')
etop_phenotypes_abba.to_excel('/Users/benh/Desktop/GI_data/phenotype_matrices/etop_phenotypes.xlsx')
keto_phenotypes_abba.to_excel('/Users/benh/Desktop/GI_data/phenotype_matrices/keto_phenotypes.xlsx')


In [57]:
## Write guide-level data

dmso1_map_guide.to_excel('/Users/benh/Desktop/GI_data/guide_maps/dmso1_sgmap.xlsx')
atr_map_guide.to_excel('/Users/benh/Desktop/GI_data/guide_maps/atr_sgmap.xlsx')
dmso2_map_guide.to_excel('/Users/benh/Desktop/GI_data/guide_maps/dmso2_sgmap.xlsx')
etop_map_guide.to_excel('/Users/benh/Desktop/GI_data/guide_maps/etop_sgmap.xlsx')
keto_map_guide.to_excel('/Users/benh/Desktop/GI_data/guide_maps/keto_sgmap.xlsx')

In [61]:
## Make phenotype matrices for all replicate1 experiments

dmso1_r1_phenotypes, dmso1_r1_singles_table, dmso1_r1_singles_phenotypes = generatePhenotypeMatrix(expt1_dmso_r1_phenotypes)
atr_r1_phenotypes, atr_r1_singles_table, atr_r1_singles_phenotypes = generatePhenotypeMatrix(expt1_atr_r1_phenotypes)
dmso2_r1_phenotypes, dmso2_r1_singles_table, dmso2_r1_singles_phenotypes = generatePhenotypeMatrix(expt2_dmso_r1_phenotypes)
etop_r1_phenotypes, etop_r1_singles_table, etop_r1_singles_phenotypes = generatePhenotypeMatrix(expt2_etop_r1_phenotypes)
keto_r1_phenotypes, keto_r1_singles_table, keto_r1_singles_phenotypes = generatePhenotypeMatrix(expt2_keto_r1_phenotypes)


/var/folders/wk/hds_px4x4qg3m4rvxqhrb0wm0000gp/T/ipykernel_60078/803172525.py:42: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for i, (sgPair, counts) in enumerate(phenotypes.sort_index().iteritems()):
/var/folders/wk/hds_px4x4qg3m4rvxqhrb0wm0000gp/T/ipykernel_60078/803172525.py:42: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for i, (sgPair, counts) in enumerate(phenotypes.sort_index().iteritems()):
/var/folders/wk/hds_px4x4qg3m4rvxqhrb0wm0000gp/T/ipykernel_60078/803172525.py:42: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for i, (sgPair, counts) in enumerate(phenotypes.sort_index().iteritems()):
/var/folders/wk/hds_px4x4qg3m4rvxqhrb0wm0000gp/T/ipykernel_60078/803172525.py:42: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for i, (sgPair, counts) in enumerate(ph

In [62]:
## Make phenotype matrices for all replicate2 experiments

dmso1_r2_phenotypes, dmso1_r2_singles_table, dmso1_r2_singles_phenotypes = generatePhenotypeMatrix(expt1_dmso_r2_phenotypes)
atr_r2_phenotypes, atr_r2_singles_table, atr_r2_singles_phenotypes = generatePhenotypeMatrix(expt1_atr_r2_phenotypes)
dmso2_r2_phenotypes, dmso2_r2_singles_table, dmso2_r2_singles_phenotypes = generatePhenotypeMatrix(expt2_dmso_r2_phenotypes)
etop_r2_phenotypes, etop_r2_singles_table, etop_r2_singles_phenotypes = generatePhenotypeMatrix(expt2_etop_r2_phenotypes)
keto_r2_phenotypes, keto_r2_singles_table, keto_r2_singles_phenotypes = generatePhenotypeMatrix(expt2_keto_r2_phenotypes)


/var/folders/wk/hds_px4x4qg3m4rvxqhrb0wm0000gp/T/ipykernel_60078/803172525.py:42: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for i, (sgPair, counts) in enumerate(phenotypes.sort_index().iteritems()):
/var/folders/wk/hds_px4x4qg3m4rvxqhrb0wm0000gp/T/ipykernel_60078/803172525.py:42: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for i, (sgPair, counts) in enumerate(phenotypes.sort_index().iteritems()):
/var/folders/wk/hds_px4x4qg3m4rvxqhrb0wm0000gp/T/ipykernel_60078/803172525.py:42: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for i, (sgPair, counts) in enumerate(phenotypes.sort_index().iteritems()):
/var/folders/wk/hds_px4x4qg3m4rvxqhrb0wm0000gp/T/ipykernel_60078/803172525.py:42: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for i, (sgPair, counts) in enumerate(ph

In [63]:
## ABBA normalize replicate1 data

dmso1_r1_phenotypes_abba, dmso1_r1_singles_phenotypes_abba = abbaAveragePhenotypes(dmso1_r1_phenotypes, dmso1_r1_singles_table)
atr_r1_phenotypes_abba, atr_r1_singles_phenotypes_abba = abbaAveragePhenotypes(atr_r1_phenotypes, atr_r1_singles_table)
dmso2_r1_phenotypes_abba, dmso2_r1_singles_phenotypes_abba = abbaAveragePhenotypes(dmso2_r1_phenotypes, dmso2_r1_singles_table)
etop_r1_phenotypes_abba, etop_r1_singles_phenotypes_abba = abbaAveragePhenotypes(etop_r1_phenotypes, etop_r1_singles_table)
keto_r1_phenotypes_abba, keto_r1_singles_phenotypes_abba = abbaAveragePhenotypes(keto_r1_phenotypes, keto_r1_singles_table)


In [64]:
## ABBA normalize replicate2 data

dmso1_r2_phenotypes_abba, dmso1_r2_singles_phenotypes_abba = abbaAveragePhenotypes(dmso1_r2_phenotypes, dmso1_r2_singles_table)
atr_r2_phenotypes_abba, atr_r2_singles_phenotypes_abba = abbaAveragePhenotypes(atr_r2_phenotypes, atr_r2_singles_table)
dmso2_r2_phenotypes_abba, dmso2_r2_singles_phenotypes_abba = abbaAveragePhenotypes(dmso2_r2_phenotypes, dmso2_r2_singles_table)
etop_r2_phenotypes_abba, etop_r2_singles_phenotypes_abba = abbaAveragePhenotypes(etop_r2_phenotypes, etop_r2_singles_table)
keto_r2_phenotypes_abba, keto_r2_singles_phenotypes_abba = abbaAveragePhenotypes(keto_r2_phenotypes, keto_r2_singles_table)


In [65]:
## Calculate replicate1 GIs

dmso1_r1_map_1, dmso1_r1_map_2, dmso1_r1_map_guide = calculateInteractions(dmso1_r1_phenotypes_abba, dmso1_r1_singles_phenotypes_abba, dmso1_r1_singles_table, quadFitForceIntercept, zstandardize=True)
atr_r1_map_1, atr_r1_map_2, atr_r1_map_guide = calculateInteractions(atr_r1_phenotypes_abba, atr_r1_singles_phenotypes_abba, atr_r1_singles_table, quadFitForceIntercept, zstandardize=True)
dmso2_r1_map_1, dmso2_r1_map_2, dmso2_r1_map_guide = calculateInteractions(dmso2_r1_phenotypes_abba, dmso2_r1_singles_phenotypes_abba, dmso2_r1_singles_table, quadFitForceIntercept, zstandardize=True)
etop_r1_map_1, etop_r1_map_2, etop_r1_map_guide = calculateInteractions(etop_r1_phenotypes_abba, etop_r1_singles_phenotypes_abba, etop_r1_singles_table, quadFitForceIntercept, zstandardize=True)
keto_r1_map_1, keto_r1_map_2, keto_r1_map_guide = calculateInteractions(keto_r1_phenotypes_abba, keto_r1_singles_phenotypes_abba, keto_r1_singles_table, quadFitForceIntercept, zstandardize=True)



In [66]:
## Calculate replicate2 GIs

dmso1_r2_map_1, dmso1_r2_map_2, dmso1_r2_map_guide = calculateInteractions(dmso1_r2_phenotypes_abba, dmso1_r2_singles_phenotypes_abba, dmso1_r2_singles_table, quadFitForceIntercept, zstandardize=True)
atr_r2_map_1, atr_r2_map_2, atr_r2_map_guide = calculateInteractions(atr_r2_phenotypes_abba, atr_r2_singles_phenotypes_abba, atr_r2_singles_table, quadFitForceIntercept, zstandardize=True)
dmso2_r2_map_1, dmso2_r2_map_2, dmso2_r2_map_guide = calculateInteractions(dmso2_r2_phenotypes_abba, dmso2_r2_singles_phenotypes_abba, dmso2_r2_singles_table, quadFitForceIntercept, zstandardize=True)
etop_r2_map_1, etop_r2_map_2, etop_r2_map_guide = calculateInteractions(etop_r2_phenotypes_abba, etop_r2_singles_phenotypes_abba, etop_r2_singles_table, quadFitForceIntercept, zstandardize=True)
keto_r2_map_1, keto_r2_map_2, keto_r2_map_guide = calculateInteractions(keto_r2_phenotypes_abba, keto_r2_singles_phenotypes_abba, keto_r2_singles_table, quadFitForceIntercept, zstandardize=True)


In [69]:
## Calculate gene level replicate1 maps

dmso1_r1_map = generateGeneMap(dmso1_r1_map_guide, dmso1_r1_singles_table)
atr_r1_map = generateGeneMap(atr_r1_map_guide, atr_r1_singles_table)
dmso2_r1_map = generateGeneMap(dmso2_r1_map_guide, dmso2_r1_singles_table)
etop_r1_map = generateGeneMap(etop_r1_map_guide, etop_r1_singles_table)
keto_r1_map = generateGeneMap(keto_r1_map_guide, keto_r1_singles_table)

In [70]:
## Calculate gene level replicate2 maps

dmso1_r2_map = generateGeneMap(dmso1_r2_map_guide, dmso1_r2_singles_table)
atr_r2_map = generateGeneMap(atr_r2_map_guide, atr_r2_singles_table)
dmso2_r2_map = generateGeneMap(dmso2_r2_map_guide, dmso2_r2_singles_table)
etop_r2_map = generateGeneMap(etop_r2_map_guide, etop_r2_singles_table)
keto_r2_map = generateGeneMap(keto_r2_map_guide, keto_r2_singles_table)

In [71]:
## Write gene-level replicate1 data

dmso1_r1_map.to_excel('/Users/benh/Desktop/GI_data/gene_maps/dmso1_r1_map.xlsx')
atr_r1_map.to_excel('/Users/benh/Desktop/GI_data/gene_maps/atr_r1_map.xlsx')
dmso2_r1_map.to_excel('/Users/benh/Desktop/GI_data/gene_maps/dmso2_r1_map.xlsx')
etop_r1_map.to_excel('/Users/benh/Desktop/GI_data/gene_maps/etop_r1_map.xlsx')
keto_r1_map.to_excel('/Users/benh/Desktop/GI_data/gene_maps/keto_r1_map.xlsx')

In [72]:
## Write gene-level replicate2 data

dmso1_r2_map.to_excel('/Users/benh/Desktop/GI_data/gene_maps/dmso1_r2_map.xlsx')
atr_r2_map.to_excel('/Users/benh/Desktop/GI_data/gene_maps/atr_r2_map.xlsx')
dmso2_r2_map.to_excel('/Users/benh/Desktop/GI_data/gene_maps/dmso2_r2_map.xlsx')
etop_r2_map.to_excel('/Users/benh/Desktop/GI_data/gene_maps/etop_r2_map.xlsx')
keto_r2_map.to_excel('/Users/benh/Desktop/GI_data/gene_maps/keto_r2_map.xlsx')

In [73]:
## Write guide-level replicate1 data

dmso1_r1_map_guide.to_excel('/Users/benh/Desktop/GI_data/guide_maps/dmso1_r1_sgmap.xlsx')
atr_r1_map_guide.to_excel('/Users/benh/Desktop/GI_data/guide_maps/atr_r1_sgmap.xlsx')
dmso2_r1_map_guide.to_excel('/Users/benh/Desktop/GI_data/guide_maps/dmso2_r1_sgmap.xlsx')
etop_r1_map_guide.to_excel('/Users/benh/Desktop/GI_data/guide_maps/etop_r1_sgmap.xlsx')
keto_r1_map_guide.to_excel('/Users/benh/Desktop/GI_data/guide_maps/keto_r1_sgmap.xlsx')

In [74]:
## Write guide-level replicate1 data

dmso1_r2_map_guide.to_excel('/Users/benh/Desktop/GI_data/guide_maps/dmso1_r2_sgmap.xlsx')
atr_r2_map_guide.to_excel('/Users/benh/Desktop/GI_data/guide_maps/atr_r2_sgmap.xlsx')
dmso2_r2_map_guide.to_excel('/Users/benh/Desktop/GI_data/guide_maps/dmso2_r2_sgmap.xlsx')
etop_r2_map_guide.to_excel('/Users/benh/Desktop/GI_data/guide_maps/etop_r2_sgmap.xlsx')
keto_r2_map_guide.to_excel('/Users/benh/Desktop/GI_data/guide_maps/keto_r2_sgmap.xlsx')

In [78]:
## Write all singles tables
dmso1_singles_table.to_excel('/Users/benh/Desktop/GI_data/singles_tables/dmso1_singles.xlsx')
atr_singles_table.to_excel('/Users/benh/Desktop/GI_data/singles_tables/atr_singles.xlsx')
dmso2_singles_table.to_excel('/Users/benh/Desktop/GI_data/singles_tables/dmso2_singles.xlsx')
etop_singles_table.to_excel('/Users/benh/Desktop/GI_data/singles_tables/etop_singles.xlsx')
keto_singles_table.to_excel('/Users/benh/Desktop/GI_data/singles_tables/keto_singles.xlsx')

dmso1_r1_singles_table.to_excel('/Users/benh/Desktop/GI_data/singles_tables/dmso1_r1_singles.xlsx')
atr_r1_singles_table.to_excel('/Users/benh/Desktop/GI_data/singles_tables/atr_r1_singles.xlsx')
dmso2_r1_singles_table.to_excel('/Users/benh/Desktop/GI_data/singles_tables/dmso2_r1_singles.xlsx')
etop_r1_singles_table.to_excel('/Users/benh/Desktop/GI_data/singles_tables/etop_r1_singles.xlsx')
keto_r1_singles_table.to_excel('/Users/benh/Desktop/GI_data/singles_tables/keto_r1_singles.xlsx')

dmso1_r2_singles_table.to_excel('/Users/benh/Desktop/GI_data/singles_tables/dmso1_r2_singles.xlsx')
atr_r2_singles_table.to_excel('/Users/benh/Desktop/GI_data/singles_tables/atr_r2_singles.xlsx')
dmso2_r2_singles_table.to_excel('/Users/benh/Desktop/GI_data/singles_tables/dmso2_r2_singles.xlsx')
etop_r2_singles_table.to_excel('/Users/benh/Desktop/GI_data/singles_tables/etop_r2_singles.xlsx')
keto_r2_singles_table.to_excel('/Users/benh/Desktop/GI_data/singles_tables/keto_r2_singles.xlsx')

In [80]:
## Write all singles phenotypes
dmso1_singles_phenotypes_abba.to_excel('/Users/benh/Desktop/GI_data/singles_phenotypes/dmso1_singles_phenotypes.xlsx')
atr_singles_phenotypes_abba.to_excel('/Users/benh/Desktop/GI_data/singles_phenotypes/atr_singles_phenotypes.xlsx')
dmso2_singles_phenotypes_abba.to_excel('/Users/benh/Desktop/GI_data/singles_phenotypes/dmso2_singles_phenotypes.xlsx')
etop_singles_phenotypes_abba.to_excel('/Users/benh/Desktop/GI_data/singles_phenotypes/etop_singles_phenotypes.xlsx')
keto_singles_phenotypes_abba.to_excel('/Users/benh/Desktop/GI_data/singles_phenotypes/keto_singles_phenotypes.xlsx')

dmso1_r1_singles_phenotypes_abba.to_excel('/Users/benh/Desktop/GI_data/singles_phenotypes/dmso1_r1_singles_phenotypes.xlsx')
atr_r1_singles_phenotypes_abba.to_excel('/Users/benh/Desktop/GI_data/singles_phenotypes/atr_r1_singles_phenotypes.xlsx')
dmso2_r1_singles_phenotypes_abba.to_excel('/Users/benh/Desktop/GI_data/singles_phenotypes/dmso2_r1_singles_phenotypes.xlsx')
etop_r1_singles_phenotypes_abba.to_excel('/Users/benh/Desktop/GI_data/singles_phenotypes/etop_r1_singles_phenotypes.xlsx')
keto_r1_singles_phenotypes_abba.to_excel('/Users/benh/Desktop/GI_data/singles_phenotypes/keto_r1_singles_phenotypes.xlsx')

dmso1_r2_singles_phenotypes_abba.to_excel('/Users/benh/Desktop/GI_data/singles_phenotypes/dmso1_r2_singles_phenotypes.xlsx')
atr_r2_singles_phenotypes_abba.to_excel('/Users/benh/Desktop/GI_data/singles_phenotypes/atr_r2_singles_phenotypes.xlsx')
dmso2_r2_singles_phenotypes_abba.to_excel('/Users/benh/Desktop/GI_data/singles_phenotypes/dmso2_r2_singles_phenotypes.xlsx')
etop_r2_singles_phenotypes_abba.to_excel('/Users/benh/Desktop/GI_data/singles_phenotypes/etop_r2_singles_phenotypes.xlsx')
keto_r2_singles_phenotypes_abba.to_excel('/Users/benh/Desktop/GI_data/singles_phenotypes/keto_r2_singles_phenotypes.xlsx')


In [84]:
dmso2_r1_singles_table

,gene
ACAD9_-_128598499.23-P1P2,ACAD9
ACAD9_-_128598508.23-P1P2,ACAD9
ACLY_-_40075090.23-P1P2,ACLY
ACLY_-_40075227.23-P1P2,ACLY
ACTR6_+_100594811.23-P1P2,ACTR6
...,...
non-targeting_01095,non-targeting
non-targeting_01213,non-targeting
non-targeting_01292,non-targeting
non-targeting_01297,non-targeting
